**420-A52-SF - Algorithmes d'apprentissage supervisé - Hiver 2021 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2021 Mikaël Swawola
<br/>
![Travaux Pratiques - Ensembles](static/19-tp-banner.png)
<br/>
**Objectif:** cette séance de travaux pratiques a pour objectif la mise en oeuvre différentes techniques d'ensembles. Le jeu de données utilisée sera **Titanic**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Exercice 1 - Chargement et préparation des données

In [ ]:
import pandas as pd

In [ ]:
titanic = pd.read_csv('../../data/titanic_train.csv', index_col='PassengerId')

In [ ]:
titanic.head()

In [ ]:
titanic['Age'].isna().sum()

In [ ]:
# Age
titanic['imp_age'] = titanic['Age'].isna()
titanic['Age'] = titanic['Age'].fillna(titanic['Age'].mean())

In [ ]:
# Embarked
titanic = pd.get_dummies(titanic, columns=['Embarked'], prefix = ['emb'], drop_first=True, dummy_na=True)

In [ ]:
# Sex
titanic['Sex'] = (titanic['Sex'] == 'female').astype(int)

In [ ]:
titanic.columns

In [ ]:
X = titanic[['Age', 'Sex', 'Pclass', 'SibSp', 'Parch', 'Fare', 'emb_Q', 'emb_S', 'imp_age']]
y = titanic['Survived']

#### Vérification de la proportion des classes positives (Survided) et négatives (Died) 

In [ ]:
y.sum()/len(y)

#### Importation de quelques librairies

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.utils.fixes import loguniform
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

#### Mise à l'échelle des données

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X_scale = scaler.transform(X)

#### Séparation des données

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_scale, y, train_size=0.7, stratify=y, random_state=2021)

## Exercice 2 - Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

[class sklearn.linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
# Grille
parameters = {'C':[1, 2, 3],
              'l1_ratio':[0, 0.5, 1]}

# Régression logistique
clf_logreg = LogisticRegression(penalty='elasticnet',
                                max_iter=10000,
                                solver='saga',
                                random_state=2021,
                                n_jobs=-1)

# GridSearch avec Validation croisée
clf_logreg_grid = GridSearchCV(clf_logreg,
                               parameters,
                               cv=5,
                               scoring="roc_auc",
                               verbose=1, n_jobs=-1).fit(X_train, y_train)

In [ ]:
clf_logreg_grid.best_score_

In [ ]:
history = {}
history['LogReg'] = {'CV': clf_logreg_grid.best_score_}
history['LogReg']['CV']

## Exercice 3 - Arbres de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier

[class sklearn.tree.DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [ ]:
# Distributions des paramètres
distributions = dict(
    criterion=['gini', 'entropy'],
    ccp_alpha=loguniform(1e-3, 1e3),
    max_depth=randint(2, 128))

# Estimateur
clf_tree = DecisionTreeClassifier(random_state=2021)
   
# Recherche aléatoire avec avec validation croisée
clf_tree_rnd = RandomizedSearchCV(clf_tree,
                                  distributions,
                                  n_iter=1000,
                                  cv=5,
                                  scoring="roc_auc",
                                  verbose=1, n_jobs=-1, random_state=2021).fit(X_train, y_train)

In [ ]:
clf_tree_rnd.best_score_

In [ ]:
history['Tree'] = {'CV': clf_tree_rnd.best_score_}
history['Tree']['CV']

## Exercice 4 - SVM

In [ ]:
from sklearn.svm import SVC

[class sklearn.svm.SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

In [ ]:
# Grille
parameters = {'kernel':['linear', 'rbf']}

# K plus proches voisins
clf_svc = SVC(probability=True,
              random_state=2021)

# GridSearch avec Validation croisée
clf_svc_grid = GridSearchCV(clf_svc,
                            parameters,
                            cv=5,
                            scoring="roc_auc",
                            verbose=1, n_jobs=-1).fit(X_train, y_train)

In [ ]:
clf_svc_grid.best_score_

In [ ]:
history['SVM'] = {'CV': clf_svc_grid.best_score_}
history['SVM']['CV']

In [ ]:
history

## Exercice 5 - VotingClassifier

[class sklearn.ensemble.VotingClassifier(estimators, voting='hard', weights=None, n_jobs=None, flatten_transform=True)](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html#sklearn.ensemble.VotingClassifier)

## Exercice 6 - Stacking

## Exercice 7 - Rajout de modèles supplémentaires au stacking (optionnel)

### Fin de l'atelier